<a href="https://colab.research.google.com/github/rajibmondal/Projects.NLP/blob/master/Bangla_ULMFiT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
  # from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# root_dir = "/content/gdrive/My Drive/"
# base_dir = root_dir + 'fastai-v3/'

In [0]:
from fastai import *
from fastai.text import *

In [40]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = '/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/'
base_dir = root_dir + 'weights/VIET_NLP'

base_dir = Path(base_dir)

# check if the dirctory exists if not make it
import os
if not (os.path.exists(base_dir)):
  base_dir.mkdir(parents=True, exist_ok=True)
base_dir

path = Path('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/')
path.ls()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/test.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/train.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki')]

In [0]:
# #from fastai import *
# from fastai.text import *
# folder = 'V_S_A'

# path = Path('/content/gdrive/My Drive/NLP/NLP_DATA/Vietnamese_sentiment_analysis_data')
# dest = path/folder
# dest.mkdir(parents=True, exist_ok=True)

In [3]:
path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/test.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/train.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki')]

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [0]:
bs = 128
# bs = 48

In [0]:
data_path = path

In [7]:
data_path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/test.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/train.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki')]

In [0]:
lang = 'vi'
#lang = 'zh'
# lang = 'bn'

In [0]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [0]:
from fastai.basics import *
import re


def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        return

    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"

    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)

    with working_directory(path):
        if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 7000 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else: f.write(l)
    f.close()
    return dest

In [11]:
get_wiki(path,lang)

extracting...


In [12]:
path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki-latest-pages-articles.xml'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/wikiextractor'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/log'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki')]

In [13]:
!head -n4 {path}/{name}

head: cannot open '/content/drive/My' for reading: No such file or directory
head: cannot open 'Drive/Project/Vietnamese_sentiment_analysis/viwiki/viwiki' for reading: No such file or directory


In [14]:
dest = split_wiki(path,lang)

0
100000
200000
300000
400000


In [15]:
dest.ls()[:15]

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Tiếng Việt.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/California.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Thụy Điển.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Thành phố Hồ Chí Minh.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Lào Cai.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Lào.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Hoa Kỳ.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Hà Giang.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki/docs/Cao Bằng.txt'),
 PosixPath('/content/drive/My Drive/Project/Vietnames

In [16]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

# data.save(f'{lang}_databunch') 
# len(data.vocab.itos),len(data.train_ds)

In [0]:
data.save('vi_wiki.pkl')

In [36]:
path.ls()

[PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/test.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/train.csv'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/weights'),
 PosixPath('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/viwiki')]

In [17]:
data.show_batch()

idx,text
0,với hơn 4 triệu xxmaj việt kiều . xxmaj tiếng xxmaj việt còn là ngôn ngữ thứ hai của các dân tộc thiểu số tại xxmaj việt xxmaj nam . xxmaj mặc dù tiếng xxmaj việt có một số từ vựng vay mượn từ tiếng xxmaj hán và trước đây dùng chữ xxmaj nôm – một hệ chữ viết dựa trên chữ xxmaj hán – để viết nhưng tiếng xxmaj
1,và là đường nối trực tiếp duy nhất cho xe cộ sang xxmaj vịnh xxmaj east . xxmaj quốc lộ xxmaj hoa xxmaj kỳ 101 nối đến điểm đầu phía tây của xxmaj xa lộ xxmaj liên tiểu bang 80 và tạo lối đi đến phía nam thành phố dọc theo xxmaj vịnh xxmaj san xxmaj francisco về phía xxmaj thung lũng xxmaj điện tử . xxmaj đi hướng bắc
2,"( trừ bang xxmaj louisiana ) . xxmaj mặc dù hiện là bộ phận của xxmaj vương quốc xxmaj liên hiệp xxmaj anh và xxmaj bắc xxmaj ireland , hệ thống tư pháp của xxmaj các toà án xxmaj anh và xxmaj wales vẫn tiếp tục với tư cách là hệ thống tư pháp độc lập với hệ thống tại xxmaj scotland . xxmaj bản chất tổng thể của luật"
3,"các dữ liệu vệ tinh hay dữ liệu của các tàu khoa học , đã tăng thêm khả năng để các nhà khoa học có thể giải thích các biến đổi trong quá khứ và trong tương lai có thể xảy ra của hệ thống xxmaj trái xxmaj đất . \n \n \n \n \n \n \n \n \n \n < / doc > \n xxbos xxmaj việt"
4,"đội tuyển bóng đá quốc gia xxmaj bỉ từng đứng thứ nhất trong bảng xếp hạng của xxup fifa vào tháng 11 năm 2015 và tháng 9 năm 2018 . \n \n xxmaj kim xxmaj xxunk và xxmaj justine xxmaj henin đều từng nhận được danh hiệu vận động viên của năm do xxmaj hiệp hội quần vợt nữ trao tặng , họ là những nữ vận động viên"


In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [20]:
learn.unfreeze()
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.861258,3.734258,0.341176,12:15


In [0]:
# mdl_path = path/'models'
# mdl_path.mkdir(exist_ok=True)

In [33]:
# learn.data.vocab.save(lm_fns.pkl)

AttributeError: ignored

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

##Vietnamese sentiment analysis

In [28]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on \nshop Đóng gói sản phẩm...,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng...,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp ...,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1


In [29]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(test_df.comment),'comment']='NA'
test_df.head()

,id,comment
0,test_000000,Chưa dùng thử nên chưa biết
1,test_000001,Không đáng tiềnVì ngay đợt sale nên mới mua n...
2,test_000002,Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắc...
3,test_000003,Vải đẹp.phom oki luôn.quá ưng
4,test_000004,Chuẩn hàng đóng gói đẹp


In [0]:
df = pd.concat([train_df, test_df], sort=False)

In [31]:
data_lm = (TextList.from_df(df, path, cols='comment')
.split_by_rand_pct(0.1, seed=42)
.label_for_lm()
.databunch(bs=bs, num_workers=1))

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, 
                                  pretrained_fnames=lm_fns,
                                  #Lm_fns pretrained model
                                  drop_mult=1.0)

In [0]:
lr = 1e-3
lr *= bs/48

In [46]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.370924,4.505360,0.284249,00:21
1,4.733423,4.384493,0.297545,00:22


In [47]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.500173,4.317554,0.305804,00:31
1,4.394874,4.201041,0.318457,00:31
2,4.279138,4.103786,0.326869,00:30
3,4.187977,4.042936,0.331459,00:31
4,4.126500,4.004142,0.335059,00:31
5,4.081538,3.983464,0.337151,00:31
6,4.053735,3.973897,0.338184,00:31
7,4.049526,3.973294,0.338267,00:31


In [0]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')


#Classifier

In [49]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [0]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [51]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [0]:
lr=2e-2
lr *= bs/48

In [53]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.438917,0.386180,0.832090,0.808268,00:03
1,0.396179,0.323812,0.853856,0.822148,00:03


In [54]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.418299,0.401769,0.807214,0.730800,00:03
1,0.398141,0.329969,0.853856,0.826770,00:03


In [55]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.371175,0.296249,0.874378,0.831908,00:04
1,0.308331,0.266029,0.894901,0.870343,00:04


In [56]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.280558,0.256742,0.899254,0.875935,00:06
1,0.252386,0.247622,0.901741,0.880389,00:06


In [57]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,time
0,0.205769,0.244990,0.904851,0.881585,00:08


In [0]:
learn_c.save(f'{lang}clas')

#Ensemble

In [0]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'{lang}clas', purge=False);

In [60]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

(tensor(0.9049), tensor(0.8868))

In [61]:
data_clas_bwd = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)
learn_c_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c_bwd.load(f'{lang}clas_bwd', purge=False);

FileNotFoundError: ignored

In [0]:
preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
accuracy(preds_b,targs_b),f1(preds_b,targs_b)

In [0]:
preds_avg = (preds+preds_b)/2

In [0]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)


    Data       
    Competition details       
    Top 3 f1 scores: 0.900, 0.897, 0.897        



In [0]:
%%time
data.save('vi_databunch.pkl')
# data.save(f'{lang}_databunch') 
# len(data.vocab.itos),len(data.train_ds)

In [0]:
# data = load_data(path, f'{lang}_databunch', bs=bs)
data = load_data(path, 'vi_databunch', bs=bs )

In [0]:
path.ls()

In [0]:
!head -n4 {path}/{name}

In [0]:
dest = split_wiki(path,lang)

In [0]:
dest.ls()[:11]

In [0]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

In [0]:
data.save()

In [0]:
len(data.vocab.itos),len(data.train_ds)

In [0]:
path.ls()

In [0]:
data = load_data(path, f'{lang}_databunch', bs=bs)

In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [0]:
path.ls()

In [0]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

In [0]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(train_df.comment),'comment']='NA'
test_df.head()

In [0]:
df = pd.concat([train_df], sort=False)

In [0]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [0]:
lr = 1e-3
lr *= bs/48

In [0]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [0]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

In [0]:
!ls

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
len(data.vocab.itos), len(data.train_ds)

In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [0]:
lr = 1e-2
lr *= bs/48

In [0]:
learn.unfreeze
learn.fit_one_cycle(1, lr, moms=(0.8, 0.7))

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/NLP"
base_dir = root_dir + 'fastai-v3/'


In [0]:
path = Path(base_dir + 'data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)
